In this notebook I'm going to sample characters for positions

In [ ]:
from collections import Counter
from multiprocessing import Pool

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
np.random.seed(42)

In [ ]:
df = pd.read_csv("/kaggle/input/bms-molecular-translation/train_labels.csv")


In [ ]:
test_df = pd.read_csv("/kaggle/input/bms-molecular-translation/sample_submission.csv")

In [ ]:
df.head()

In [ ]:
# let's see shortest and longest strings
df_lengths = df.InChI.apply(len)
print(df_lengths.max())
print(df_lengths.min())

In [ ]:
df_lengths.plot(kind="hist")

In [ ]:
positions_freqs = [Counter() for _ in range(df_lengths.max())]
for inch in df.InChI:
    for pos, char in enumerate(inch):
        positions_freqs[pos][char] += 1

In [ ]:
len(positions_freqs)

In [ ]:
positions_freqs[0]

In [ ]:
positions_freqs[10]

In [ ]:
positions_freqs[20]

In [ ]:
cnts = np.array([v for v in positions_freqs[20].values()])
sum(cnts / sum(cnts)

In [ ]:
df_inchi_lens_freqs = df_lengths.value_counts(normalize=True)
def sample_length(size=1):
    return np.random.choice(a=df_inchi_lens_freqs.index.to_list(), p=df_inchi_lens_freqs.values.tolist(), size=size)
sample_length(4)

In [ ]:
pos2char_proba = []
for cnt in positions_freqs:
    counts = np.array(list(cnt.values()))
    freqs = counts / sum(counts)
    sampl_chars = list(cnt)
    pos2char_proba.append((sampl_chars, freqs))

In [ ]:
def sample_characters(leng: int):
    chars = []
    for i in range(leng):
        sampl_chars, freqs = pos2char_proba[i]
        chars.append(np.random.choice(sampl_chars, p=freqs))
    return "".join(chars)

sample_characters(400)

This is ridiculous, but okay for baseline

In [ ]:
test_df.shape

In [ ]:
test_df.head()

In [ ]:
test_lengths = sample_length(test_df.shape[0])

In [ ]:
result = []
with Pool(16) as pool:
    for res in tqdm(pool.imap(sample_characters, test_lengths, chunksize=60), total=test_lengths.shape[0]):
        result.append(res)

In [ ]:
test_df["InChI"] = result

In [ ]:
test_df.to_csv("output.csv", index=False)